<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/FINALtraining013397.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk tqdm

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import SGD
#from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import re
import nltk
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Load the dataset
with open('/content/dataset3.json') as f:
    intents = json.load(f)

from keras.preprocessing.text import Tokenizer

# Preprocess Data - remove stopwords, tokenize, and lemmatize
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
tokenizer = Tokenizer()

data = []
labels = []

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # Improvements
        pattern = pattern.lower().strip()
        tokenized_pattern = nltk.word_tokenize(pattern)
        tagged_words = nltk.pos_tag(tokenized_pattern)
        nouns_verbs = [word for word, tag in tagged_words if tag in ['NN','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
        filtered_words = [lemmatizer.lemmatize(word) for word in nouns_verbs]

        sequence = ' '.join(filtered_words)
        data.append(sequence)
        labels.append(intent['tag'])

# Create a Pandas DataFrame
df = pd.DataFrame({'Patterns': data, 'Tags': labels})

# Encode the Labels by converting tags into numerical form
encoder = LabelEncoder()
df['Tags'] = encoder.fit_transform(df['Tags'])
df['Tags'] = to_categorical(df['Tags'])

vocabulary_size = len(tokenizer.word_index) + 1
print("Vocabulary Size:", vocabulary_size)

In [ ]:
import pickle
pickle.dump(data, open('words.pkl', 'wb'))
pickle.dump(labels, open('labels.pkl', 'wb'))

In [ ]:
# Calculate the average sequence length
sequences_lengths = [len(sequence.split()) for sequence in df['Patterns']]
average_sequence_length = sum(sequences_lengths) / len(sequences_lengths)

# Set max_sequence_length based on the average length
multiplier = 2
max_sequence_length = int(average_sequence_length + multiplier * average_sequence_length)

# Print or use the calculated max_sequence_length
print("Average Sequence Length:", average_sequence_length)
print("Estimated Max Sequence Length:", max_sequence_length)


Average Sequence Length: 17.613587047237523
Estimated Max Sequence Length: 52


In [ ]:
df

In [ ]:
import collections
word_counts = collections.Counter(data)
num_words = len(word_counts)

print(f"The number of unique words in the vocabulary is {num_words}")

The number of unique words in the vocabulary is 5649


PERFORM FEATURE EXTRACTION USING TDIF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# The CountVectorizer converts the text data into numerical vectors by counting the occurrence of each word in the text
vectorizer = CountVectorizer()
vectorizer.fit(data)
X = vectorizer.transform(data)  # Transform text to feature vectors
y = df['Tags']

# Print X and y to see if shapes match
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (5991, 8682)
y shape: (5991,)


In [ ]:
# Split the data into training and temporary (validation + test) sets
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the temporary set into validation and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)

# Check the shapes of the resulting sets
print("Train set shapes - X: {}, y: {}".format(X_train.shape, y_train.shape))
print("Validation set shapes - X: {}, y: {}".format(X_val.shape, y_val.shape))
print("Test set shapes - X: {}, y: {}".format(X_test.shape, y_test.shape))


Train set shapes - X: (3594, 8682), y: (3594,)
Validation set shapes - X: (1198, 8682), y: (1198,)
Test set shapes - X: (1199, 8682), y: (1199,)


In [169]:
num_classes = 1
max_sequence_length = 52

In [170]:
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=100, input_length= 8682))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, kernel_regularizer='l2'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

In [171]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 8682, 100)         884900    
                                                                 
 lstm_32 (LSTM)              (None, 8682, 256)         365568    
                                                                 
 lstm_33 (LSTM)              (None, 128)               197120    
                                                                 
 dropout_32 (Dropout)        (None, 128)               0         
                                                                 
 dense_32 (Dense)            (None, 64)                8256      
                                                                 
 dropout_33 (Dropout)        (None, 64)                0         
                                                                 
 dense_33 (Dense)            (None, 1)               

In [172]:
from sklearn.preprocessing import LabelEncoder

# Assuming your labels are in 'y_train' and 'y_val'
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_val)


In [173]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [174]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [175]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train_encoded.shape)

X_train shape: (3594, 8682)
y_train shape: (3594,)


In [176]:
hist = model.fit(X_train, y_train_encoded, epochs=20, validation_data=(X_val, y_val_encoded), callbacks=[early_stop])

Epoch 1/20
113/113 [==============================] - 100s 821ms/step - loss: 0.9322 - accuracy: 0.9858 - val_loss: 0.1008 - val_accuracy: 0.9875
Epoch 2/20
113/113 [==============================] - 92s 813ms/step - loss: 0.0881 - accuracy: 0.9872 - val_loss: 0.0706 - val_accuracy: 0.9875
Epoch 3/20
 29/113 [======>.......................] - ETA: 1:00 - loss: 0.0684 - accuracy: 0.9903

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
def plot(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    plot(hist)

In [ ]:
model.save("chatbot_model.h5")
print('Done')

In [ ]:
predictions = model.predict(X_test)

# Get predicted labels
predicted_labels = np.argmax(predictions, axis=1)

true_labels = y_test


# Calculate accuracy
accuracy = np.sum(predicted_labels == true_labels) / len(true_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming val_true_labels and val_pred_idx are NumPy arrays or lists
val_true_labels = y_val_encoded  # Use y_val directly as true labels
val_pred_idx = np.argmax(model.predict(X_val), axis=-1)

# Check if val_true_labels is an array-like object
if isinstance(val_true_labels, np.ndarray) and val_true_labels.ndim > 0:
    # Calculate accuracy and other metrics
    accuracy_score_value = accuracy_score(val_true_labels, val_pred_idx)
    precision = precision_score(val_true_labels, val_pred_idx, average='weighted', zero_division=1)
    recall = recall_score(val_true_labels, val_pred_idx, average='weighted', zero_division=1)
    f1 = f1_score(val_true_labels, val_pred_idx, average='weighted')
    confusion_mat = confusion_matrix(val_true_labels, val_pred_idx)

    # Print the results
    print(f"Accuracy: {accuracy_score_value:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print('Confusion Matrix:')
    print(confusion_mat)
else:
    print("Error: val_true_labels is not an array-like object.")

# Print additional information for debugging
print("Shape of y_val:", y_val.shape)
print("Unique values in y_val:", np.unique(y_val))


In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)